In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 31 19:47:37 2020

@author: MsTal
"""
import numpy as np
import scipy
import scipy.stats as si
from scipy.stats import norm
import math

def euro_vanilla_put(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    put = (K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0) - S * si.norm.cdf(-d1, 0.0, 1.0))
    
    return put

def rsi(P,wind):
    delta = P.diff()
    window = wind
    up_days = delta.copy()
    up_days[delta<=0]=0.0
    down_days = abs(delta.copy())
    down_days[delta>0]=0.0
    RS_up = up_days.rolling(window).mean()
    RS_down = down_days.rolling(window).mean()
    rsi= 100-100/(1+RS_up/RS_down)
    return rsi

def euro_vanilla_call(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

def getWeights(d,lags):
    # return the weights from the series expansion of the differencing operator
    # for real orders d and up to lags coefficients
    w=[1]
    for k in range(1,lags):
        w.append(-w[-1]*((d-k+1))/k)
    w=np.array(w).reshape(-1,1) 
    return w


def ts_differencing(series, order, lag_cutoff):
    # return the time series resulting from (fractional) differencing
    # for real orders order up to lag_cutoff coefficients
    
    weights=getWeights(order, lag_cutoff)
    res=0
    for k in range(lag_cutoff):
        res += weights[k]*series.shift(k).fillna(0)
    return res[lag_cutoff:] 

def delta_call(S, K, T, r, sigma):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    delta_call = si.norm.cdf(d1, 0.0, 1.0)
    
    return delta_call

def delta_put(S, K, T, r, sigma):
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    delta_put = si.norm.cdf(-d1, 0.0, 1.0)
    return -delta_put

def To_Annual(P_P,Per): # take time seires and reduce frequancy
    import pandas as pd
    BTResults = pd.DataFrame()
    BTResults['MyDate'] = pd.to_datetime(P_P.index)
    BTResults.index=P_P.index
    BTResults['P']=P_P.values
    BTResults['DD']=BTResults.P/BTResults.P.expanding(min_periods=2).max()-1
    if Per=='month':
        Years=pd.DatetimeIndex(BTResults['MyDate']).month
    elif Per=='week':
        Years=pd.DatetimeIndex(BTResults['MyDate']).week
    else:
        Years=pd.DatetimeIndex(BTResults['MyDate']).year
        
    Inx=[]
    count=0
    Inx.append(0)
    for Y in Years:
        if count>0:
            if Years[count-1]!=Years[count]:
                Inx.append(count-1)
        count=count+1
    Inx.append(count-1)
        
    return BTResults.iloc[Inx]
    

def Bollinger(P,Wind,wide):
    import pandas as pd
    data = pd.DataFrame()
    data['P']=P
    data['bb_bbm'] = P.rolling(window=Wind).mean()
    
    STD=data.P.rolling(window=Wind).std()
    
    data['bb_bbh'] = data['bb_bbm'] + abs(wide*STD)
    data['bb_bbl'] = data['bb_bbm'] - abs(wide*STD )
    
    return data


def delta_to_strike(S0,delta,T,sigma,r,right='C'):
    if right=='P':
        strike=S0 * math.exp(norm.ppf(delta * math.exp((r)*T) ) * sigma * math.sqrt(T) + ((sigma**2)/2) * T)
    elif right=='C':
        strike=S0 * math.exp(-norm.ppf(delta * math.exp((r)*T) ) * sigma * math.sqrt(T) + ((sigma**2)/2) * T)
    else:
        print('set a valid right C or P')
        strike=None
    return strike